# Задание №2. Парсинг HH.ru + предобработка текстовых данных
Используя публичный API сайта HeadHunter (особое внимание на метод https://api.hh.ru/vacancies/<id>) необходимо:

- скачать минимум 200 вакансий по заданному ключевому слову (можете задать слова, характеризующие профессии в IT); если вакансий менее 200 - скачать все, что есть;

- предобработать каждую вакансию (очистить от тегов и специальных символов, токенизировать, нормализовать любым из двух способов, удалить стоп-слова);

- сохранить все полученные вакансии (список списков токенов) на диск (Google Drive) с помощью pickle.

In [45]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [46]:
#pip install pymorphy3

In [47]:
import requests
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re
from pymorphy3 import MorphAnalyzer
import string
import numpy as np
import pickle

In [48]:
morph = MorphAnalyzer()
stopwords = stopwords.words("russian")


def preProccesingData(text):
    text_no_html = re.sub('<[^>]*>', '', text)
    only_letters = re.sub("[^\s^\w]+", " ", text_no_html)
    tokenized = word_tokenize(only_letters)
    normalized = [morph.normal_forms(word)[0] for word in tokenized]
    no_stop_words = [
        word
        for word in normalized
        if (
            (word not in string.punctuation)
            and (word not in stopwords)
            and word.isalpha()
        )
    ]
    return no_stop_words

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [50]:
def fetch_vacancies(keyword, max_vacancies=200):
    vacancies = []
    page = 0

    while len(vacancies) < max_vacancies:
        response = requests.get(
            "https://api.hh.ru/vacancies", params={"text": keyword, "page": page}
        )
        data = response.json()
        page += 1

        if not data["items"]:
            break

        for item in data["items"]:
            vacancy_id = item["id"]
            vacancy_response = requests.get(f"https://api.hh.ru/vacancies/{vacancy_id}")
            vacancy_data = vacancy_response.json()
            description = vacancy_data.get("description", "")

            # Предобрабатываем описание
            tokens = preProccesingData(description)
            vacancies.append(tokens)

            if len(vacancies) >= max_vacancies:
                break

    return vacancies

In [51]:
keyword = "Тестировщик"
vacancies = fetch_vacancies(keyword)

In [52]:
vacancies[0]

['предлагать',
 'стажировка',
 'frontend',
 'react',
 'backend',
 'java',
 'spring',
 'разработчик',
 'существующий',
 'проект',
 'получение',
 'опыт',
 'коммерческий',
 'разработка',
 'готовый',
 'рассмотреть',
 'также',
 'кандидат',
 'навык',
 'бизнес',
 'анализ',
 'также',
 'qa',
 'дизайнер',
 'предложение',
 'мочь',
 'полезно',
 'данный',
 'момент',
 'работать',
 'либо',
 'иметь',
 'время',
 'желание',
 'трудиться',
 'команда',
 'развивать',
 'свой',
 'навык',
 'помогать',
 'развиваться',
 'работа',
 'проект',
 'команда',
 'помочь',
 'быстрый',
 'найти',
 'оплачивать',
 'работа',
 'иметь',
 'преимущество',
 'сравнение',
 'самостоятельный',
 'развитие',
 'ваш',
 'трудоустройство',
 'данный',
 'момент',
 'накопление',
 'опыт',
 'месяц',
 'год',
 'важный',
 'преимущество',
 'резюме',
 'описание',
 'проект',
 'веб',
 'приложение',
 'планирование',
 'задача',
 'данный',
 'момент',
 'приносить',
 'доход',
 'разрабатываться',
 'исключительно',
 'сила',
 'стажёр',
 'разрабатываться',
 'око

In [53]:
with open('/content/drive/MyDrive/vacancies_tokens.pkl', 'wb') as f:
        pickle.dump(vacancies, f)